In [1]:
import math
import requests
import numpy as np
import pandas as pd
import re
import time

from urllib.parse import unquote, urlencode, quote_plus
from datetime import datetime, date, timedelta

from joblib import dump, load

# Plots
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst"

serviceKey = "wGokgRxD1t3z5G4u7MsWumpoCeiWO8JM6yZ87rX1ELTO9nMSUuMOQjHj70rAzuopgyB1iLdKX0S9WK0RLs88bQ==" # 공공데이터 포털에서 생성된 본인의 서비스 키를 복사 / 붙여넣기
serviceKeyDecoded = unquote(serviceKey, 'UTF-8') # 공데이터 포털에서 제공하는 서비스키는 이미 인코딩된 상태이므로, 디코딩하여 사용해야 함

while True:
  now = datetime.now()
  today = datetime.today().strftime("%Y%m%d")
  y = date.today() - timedelta(days=1)
  yesterday = y.strftime("%Y%m%d")

  if now.minute<45: # base_time와 base_date 구하는 함수
      if now.hour==0:
          base_time = "2330"
          base_date = yesterday
      else:
          pre_hour = now.hour-1
          if pre_hour<10:
              base_time = "0" + str(pre_hour) + "30"
          else:
              base_time = str(pre_hour) + "30"
          base_date = today
  else:
      if now.hour < 10:
          base_time = "0" + str(now.hour) + "30"
      else:
          base_time = str(now.hour) + "30"
      base_date = today

  queryParams = '?' + urlencode({ quote_plus('serviceKey') : serviceKeyDecoded, quote_plus('base_date') : base_date, quote_plus('pageNo') : 1,
                                      quote_plus('base_time') : base_time, quote_plus('nx') : 98, quote_plus('ny') : 76,
                                      quote_plus('dataType') : 'json', quote_plus('numOfRows') : '60'}) #페이지로 안나누고 한번에 받아오기 위해 numOfRows=60으로 설정해주었다


      # 값 요청 (웹 브라우저 서버에서 요청 - url주소와 파라미터)
  res = requests.get(url + queryParams, verify=False) # verify=False이거 안 넣으면 에러남ㅜㅜ
  items = res.json().get('response').get('body').get('items') #데이터들 아이템에 저장



  # 저장된 모델 로드
  stacking = load('stacking_model.joblib')

  data = {
      '기온(°C)': [float(items['item'][3]['obsrValue'])],
      '풍향(deg)': [float(items['item'][5]['obsrValue'])],
      '풍속(m/s)': [float(items['item'][7]['obsrValue'])],
      '강수량(mm)': [float(items['item'][2]['obsrValue'])]
  }

  new_data = pd.DataFrame(data)

  predictions = stacking.predict(new_data)

  probabilities = stacking.predict_proba(new_data)

  print(data)
  print(f"침수확률 : {probabilities[0][1]:.2%}")

  time.sleep(600)

c:\DEV\Langs\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: 'stacking_model.joblib'